In [1]:
import pandas as pd
import os

# Benchmarks

In [5]:
path_to_results = os.getcwd() + "/results/"


index_pd = pd.MultiIndex.from_tuples(
    [
        ("RUMBoost", "Constant"),
        ("RUMBoost", "Linear"),
        # ("RUMBoost", "Cubic"),
        ("RUMBoost", "Constant + linear"),
        # ("RUMBoost", "Constant + cubic"),
        # ("APLR", "Linear"),
    ]
)

columns_pd = pd.MultiIndex.from_product([("LPMC", "SwissMetro"), ("Monotonic", "Non-monotonic")])

benchmark_table_test = pd.DataFrame({},
    columns=columns_pd,
    index=index_pd,
)
benchmark_table_train = pd.DataFrame({},
    columns=columns_pd,
    index=index_pd,
)
benchmark_table_ct = pd.DataFrame({},
    columns=columns_pd,
    index=index_pd,
)

for model in ["RUMBoost"]:#, "APLR"]:
    for monotone in ["True", "False"]:
        for model_type in ["constant","linear", "constant_linear"]:#,"linear", "cubic","constant_cubic"]:
            for dataset in ["LPMC", "SwissMetro"]:
                if model == "APLR" and model_type != "linear":
                    continue
                try:
                    results = pd.read_csv(
                        f"{path_to_results}{dataset}/{model}/{model_type}/mono{monotone}/results_True.csv"
                    )
                except FileNotFoundError:
                    print(f"Results file not found for {dataset}, {model}, {model_type}, monotone={monotone}")
                    continue
                monotone_c = "Monotonic" if monotone == "True" else "Non-monotonic"
                model_type_c = "Constant" if model_type == "constant" else "Linear" if model_type == "linear" else "Cubic" if model_type == "cubic" else "Constant + linear" if model_type == "constant_linear" else "Constant + cubic"
                benchmark_table_test.loc[(model, model_type_c), (dataset, monotone_c)] = results["loss_test"].values[0].round(4)
                benchmark_table_train.loc[(model, model_type_c), (dataset, monotone_c)] = results["train_loss"].values[0].round(4)
                benchmark_table_ct.loc[(model, model_type_c), (dataset, monotone_c)] = results["train_time"].values[0].round(4)


Results file not found for LPMC, RUMBoost, linear, monotone=True
Results file not found for LPMC, RUMBoost, linear, monotone=False


In [6]:
benchmark_table_test

LPMC               SwissMetro              
                           Monotonic Non-monotonic  Monotonic Non-monotonic
RUMBoost Constant             0.6741        0.6745     0.7675        0.7758
         Linear                  NaN           NaN     0.8387        0.8345
         Constant + linear    0.6729        0.6732     0.7675        0.7759

In [7]:
benchmark_table_test.to_latex(
    f"{path_to_results}benchmark_table_test.tex",)

In [8]:
benchmark_table_train

LPMC               SwissMetro              
                           Monotonic Non-monotonic  Monotonic Non-monotonic
RUMBoost Constant             0.6397        0.6359     0.6704        0.6373
         Constant + linear    0.6405        0.6363     0.6712        0.6326

In [9]:
benchmark_table_ct

LPMC               SwissMetro              
                            Monotonic Non-monotonic  Monotonic Non-monotonic
RUMBoost Constant            166.7323      226.9962    36.1314      336.7697
         Constant + linear  4367.3497     6527.3914     75.476        778.45